In [3]:
import pandas as pd
import pandas_profiling
import math
import requests
import requests_cache
from IPython.display import Image, display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

### Data dictionary: https://www.arcgis.com/sharing/rest/content/items/70392a096a8e431381f1f692aaa06afd/info/metadata/metadata.xml?format=default&output=html

In [5]:
crashes = pd.read_csv('../data/Crashes_in_DC.csv', low_memory=False)

In [6]:
crashes.head()

,X,Y,OBJECTID,CRIMEID,CCN,REPORTDATE,ROUTEID,MEASURE,OFFSET,STREETSEGID,...,MPDLATITUDE,MPDLONGITUDE,MPDGEOX,MPDGEOY,BLOCKKEY,SUBBLOCKKEY,FATALPASSENGER,MAJORINJURIESPASSENGER,MINORINJURIESPASSENGER,UNKNOWNINJURIESPASSENGER
0,-77.001648,38.965057,167334057,27711880,18099907,2018/06/20 02:04:02+00,12031982,8872.07,10.20,1547.0,...,38.964979,-77.001721,399817.66,144131.79,e84f893b8e56347a75ae5392f48f81ff,e84f893b8e56347a75ae5392f48f81ff,0,0,0,0
1,-77.086496,38.960196,167334058,27974025,19039781,2019/03/09 02:51:11+00,11092692,1721.13,5.89,NaN,...,38.960226,-77.086542,392393.41,143414.76,ff0c117e444eb7fd7f04bc107e733555,ff0c117e444eb7fd7f04bc107e733555,0,0,0,0
2,-76.948891,38.885685,167334059,27363032,17116467,2017/07/08 02:48:56+00,13075662,586.84,0.01,218.0,...,38.885677,-76.948889,404434.46,135316.58,04e13c1161aeef67f9a611e13b77152a,ee46ecb5c1661b1efeb61ad81907f753,0,0,0,0
3,-77.015903,38.882372,167334060,27842957,18183059,2018/10/28 22:39:00+00,15048473,0.00,5.68,6364.0,...,38.882313,-77.015901,398763.04,136862.91,3dda6f37c0286a68bc3077286332de14,2f9c18cba8e17da1af636842ea15cd37,0,0,0,0
4,-77.077044,38.933496,167334061,25090446,14121005,2014/08/12 05:00:00+00,11057982,1898.02,0.32,827.0,...,38.933491,-77.077042,NaN,NaN,5ec46695e6cd293fc09d022f0312f926,3e804f84b5130e8a9fdc6f44387d22a2,0,0,0,0


In [7]:
crashes.columns

Index(['X', 'Y', 'OBJECTID', 'CRIMEID', 'CCN', 'REPORTDATE', 'ROUTEID',
       'MEASURE', 'OFFSET', 'STREETSEGID', 'ROADWAYSEGID', 'FROMDATE',
       'TODATE', 'MARID', 'ADDRESS', 'LATITUDE', 'LONGITUDE', 'XCOORD',
       'YCOORD', 'WARD', 'EVENTID', 'MAR_ADDRESS', 'MAR_SCORE',
       'MAJORINJURIES_BICYCLIST', 'MINORINJURIES_BICYCLIST',
       'UNKNOWNINJURIES_BICYCLIST', 'FATAL_BICYCLIST', 'MAJORINJURIES_DRIVER',
       'MINORINJURIES_DRIVER', 'UNKNOWNINJURIES_DRIVER', 'FATAL_DRIVER',
       'MAJORINJURIES_PEDESTRIAN', 'MINORINJURIES_PEDESTRIAN',
       'UNKNOWNINJURIES_PEDESTRIAN', 'FATAL_PEDESTRIAN', 'TOTAL_VEHICLES',
       'TOTAL_BICYCLES', 'TOTAL_PEDESTRIANS', 'PEDESTRIANSIMPAIRED',
       'BICYCLISTSIMPAIRED', 'DRIVERSIMPAIRED', 'TOTAL_TAXIS',
       'TOTAL_GOVERNMENT', 'SPEEDING_INVOLVED', 'NEARESTINTROUTEID',
       'NEARESTINTSTREETNAME', 'OFFINTERSECTION', 'INTAPPROACHDIRECTION',
       'LOCATIONERROR', 'LASTUPDATEDATE', 'MPDLATITUDE', 'MPDLONGITUDE',
       'MPDGEOX', 'MPD

In [8]:
crashes.describe()

,X,Y,OBJECTID,CRIMEID,MEASURE,OFFSET,STREETSEGID,ROADWAYSEGID,MARID,LATITUDE,...,SPEEDING_INVOLVED,OFFINTERSECTION,MPDLATITUDE,MPDLONGITUDE,MPDGEOX,MPDGEOY,FATALPASSENGER,MAJORINJURIESPASSENGER,MINORINJURIESPASSENGER,UNKNOWNINJURIESPASSENGER
count,237798.000000,237798.000000,2.377980e+05,2.377980e+05,237798.000000,237798.000000,208312.000000,208312.000000,237798.000000,237788.000000,...,237798.000000,237798.000000,171017.000000,171017.000000,117852.000000,117852.000000,237798.000000,237798.000000,237798.000000,237798.000000
mean,-76.478040,38.531809,1.674530e+08,2.670723e+07,1658.208030,12.341329,5421.476310,15563.016845,285889.912127,38.904862,...,0.017124,25.477912,38.385044,-76.270864,399441.091269,137243.427111,0.000332,0.023894,0.079231,0.009180
std,5.998522,4.209904,6.864651e+04,1.201168e+06,1741.326452,13.591661,4662.976219,11257.211048,227170.277955,0.030762,...,0.129830,42.684429,4.956520,7.052731,3139.173416,3467.594722,0.019125,0.303307,0.377024,0.130423
min,-78.815503,-9.000001,1.673341e+08,2.341134e+07,0.000000,0.000000,-9.000000,0.000000,1.000000,38.805520,...,0.000000,0.000000,-9.000000,-78.815500,390097.460000,127294.260000,0.000000,0.000000,0.000000,0.000000
25%,-77.032205,38.887704,1.673935e+08,2.652628e+07,365.430000,0.050000,299.000000,6298.000000,217995.000000,38.889089,...,0.000000,0.660000,38.886183,-77.031710,397307.480000,135393.110000,0.000000,0.000000,0.000000,0.000000
50%,-77.010773,38.904448,1.674530e+08,2.683051e+07,1142.120000,6.860000,5054.500000,12863.000000,272758.000000,38.904688,...,0.000000,14.070000,38.903641,-77.009425,399192.440000,137302.150000,0.000000,0.000000,0.000000,0.000000
75%,-76.984848,38.922456,1.675124e+08,2.765500e+07,2352.100000,21.670000,9459.000000,23624.250000,301153.250000,38.922661,...,0.000000,33.170000,38.920766,-76.984514,401465.930000,139214.120000,0.000000,0.000000,0.000000,0.000000
max,77.012717,40.000008,1.675719e+08,2.855307e+07,10750.110000,50.000000,15824.000000,40635.000000,914195.000000,38.994746,...,2.000000,973.450000,40.000000,76.997665,407828.720000,147354.210000,2.000000,51.000000,17.000000,15.000000


In [9]:
profile = pandas_profiling.ProfileReport(crashes)

In [10]:
# Generate Summary Statistics Report (takes a few minutes)
profile.to_widgets()

Summarize dataset:  89%|████████▉ | 66/74 [22:04<02:40, 20.07s/it, Calculate cramers correlation] 


KeyboardInterrupt: 

## Demonstrate ability to take a single crash entry and plot it on a map:

In [27]:
# Enter your mapbox token or load it from environment variable here
mapbox_token = os.getenv('MAPBOX_TOKEN')

In [28]:
# Code pulled from Andrew's work in repo:
requests_cache.install_cache('satellite_cache')
ZOOM_LEVEL = 18


def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)


def get_tile(lat, lon, token=mapbox_token, zoom=ZOOM_LEVEL):
    x, y = deg2num(lat, lon, zoom)
    r = requests.get(f"https://api.mapbox.com/v4/mapbox.satellite/{zoom}/{x}/{y}@2x.jpg70", params={
        'access_token': token
    })
    if r.status_code == 200:
        return {'url': r.url, 'zoom': zoom, 'lat': lat, 'lon': lon}
    else:
        print(f"FAILURE {r.status_code}")
        

def deg2percentile(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xpercentile = math.modf((lon_deg + 180.0) / 360.0 * n)[0]
    ypercentile = math.modf((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)[0]
    return (xpercentile, ypercentile)


def draw_square_at_percentile(img, x_pct, y_pct):
    """Adds a square over the specified percentile location
    # https://stackoverflow.com/questions/55545400/how-to-draw-a-point-in-an-image-using-given-coordinate"""
    red = [255, 0, 0]
    img = img.copy()
    height, width, bands = img.shape

    # Update red pixel value for RGBA image
    if bands == 4:
        red = [1, 0, 0, 1]

    # Turn percentiles into pixels:
    x_pixel = round(x_pct * width)
    y_pixel = round(y_pct * height)

    # Draw a 10x10 red square at pixel location
    for i in range(y_pixel-5, y_pixel+5):
        for j in range(x_pixel-5, x_pixel+5):
            try:
                img[i][j] = red
            except IndexError:
                # Box hits edge of image
                continue
            
    return img

In [29]:
# Use the 5th crash in the table
crash = crashes.iloc[4]
lat = crash['LATITUDE']
lon = crash['LONGITUDE']
img_urls = [get_tile(lat, lon, zoom=zoom) for zoom in [18, 17, 16]]

In [31]:
for img_url in img_urls:
    img = mpimg.imread(img_url['url'], format='jpg')
    x_pct, y_pct = deg2percentile(img_url['lat'], img_url['lon'], img_url['zoom'])
    img = draw_square_at_percentile(img, x_pct, y_pct)
    # Update figure size based on image size
    height, width, bands = img.shape
    dpi = 80
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    figure = plt.figure(figsize=figsize)
    axes = figure.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    axes.axis('off')
    axes.imshow(img)

AttributeError: 'AxesImage' object has no property 'width'

38.964946000000005


In [49]:
# TODO: DC crash data to GeoJSON
# crashes[['LATITUDE', 'LONGITUDE']]
points = []
for k, row in crashes[['LATITUDE', 'LONGITUDE']].iterrows():    
    points.append(
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row['LONGITUDE'], row['LATITUDE']]
            },
            "properties": {
                "name": "incident"
            }
        }
    )

In [50]:
obj = {
    "type": "FeatureCollection",
    "features": points
}

In [51]:
import json
with open('dcincidents.geojson', 'w') as outfile:
    json.dump(obj, outfile)